# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

### Limpieza de Datos

Primero vamos a cargar todos nuestros datos:

In [0]:
df1 = spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("header", "true") \
    .load("dbfs:/FileStore/pyspark-exam/case_3/dataset/airbnb_last_review.tsv")

df2 = spark.read.format("csv") \
    .option("header", "true") \
    .load("dbfs:/FileStore/pyspark-exam/case_3/dataset/airbnb_price.csv")

df3 = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .load("dbfs:/FileStore/pyspark-exam/case_3/dataset/airbnb_room_type.csv")

In [0]:
df1.show()
df2.show()
df3.show()

+----------+----------------+---------------+
|listing_id|       host_name|    last_review|
+----------+----------------+---------------+
|      2595|        Jennifer|    May 21 2019|
|      3831|     LisaRoxanne|   July 05 2019|
|      5099|           Chris|   June 22 2019|
|      5178|        Shunichi|   June 24 2019|
|      5238|             Ben|   June 09 2019|
|      5295|            Lena|   June 22 2019|
|      5441|            Kate|   June 23 2019|
|      5803|          Laurie|   June 24 2019|
|      6021|         Claudio|   July 05 2019|
|      6848|   Allen & Irina|   June 29 2019|
|      7097|            Jane|   June 28 2019|
|      7322|            Doti|   July 01 2019|
|      7726|Adam And Charity|   June 22 2019|
|      8024|           Lisel|   July 01 2019|
|      8025|           Lisel|January 01 2019|
|      8110|           Lisel|   July 02 2019|
|      8490|        Nathalie|   June 19 2019|
|      8505|         Gregory|   June 23 2019|
|      9518|            Shon|   Ju

Ahora para manejar todos los datos a la vez, vamos a unir nuestros datasets en uno solo, se puede ver que todos tienen una correspondencia con el campo `listing_id`, así que este sera nuestro punto de partida.

In [0]:
# Uninos el primer dataframe con el segundo.
df_airbnb = df1.join(df2, on="listing_id", how="inner")

# Luego unimos ese dataframe con el tercero
df_airbnb = df_airbnb.join(df3, on="listing_id", how="inner")

# Vemos si se fusionó correctamente.
df_airbnb.show()

+----------+----------------+---------------+-----------+--------------------+--------------------+---------------+
|listing_id|       host_name|    last_review|      price|         nbhood_full|         description|      room_type|
+----------+----------------+---------------+-----------+--------------------+--------------------+---------------+
|      2595|        Jennifer|    May 21 2019|225 dollars|  Manhattan, Midtown|Skylit Midtown Ca...|Entire home/apt|
|      3831|     LisaRoxanne|   July 05 2019| 89 dollars|Brooklyn, Clinton...|Cozy Entire Floor...|Entire home/apt|
|      5099|           Chris|   June 22 2019|200 dollars|Manhattan, Murray...|Large Cozy 1 BR A...|Entire home/apt|
|      5178|        Shunichi|   June 24 2019| 79 dollars|Manhattan, Hell's...|Large Furnished R...|   private room|
|      5238|             Ben|   June 09 2019|150 dollars|Manhattan, Chinatown|Cute & Cozy Lower...|Entire home/apt|
|      5295|            Lena|   June 22 2019|135 dollars|Manhattan, Uppe

Ahora que tenemos todo en una sola, vamos a tener que formatear correctamente los datos, primero vemos la estructura de nuestro dataframe.

In [0]:
df_airbnb.printSchema()

root
 |-- listing_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- price: string (nullable = true)
 |-- nbhood_full: string (nullable = true)
 |-- description: string (nullable = true)
 |-- room_type: string (nullable = true)



Ahora cambiaremos los tipos de datos segun corresponda.

In [0]:
from pyspark.sql.functions import col, regexp_replace, lower, to_date

# Eliminar la palabra "dollars" y convertir el precio a un tipo numérico (float)
df_airbnb = df_airbnb.withColumn(
    "price",
    regexp_replace(col("price"), " dollars", "").cast("float")
)

# Poner todos los tipos de habitaciones en minúsculas
df_airbnb = df_airbnb.withColumn("room_type", lower(col("room_type")))

# Convertir la fecha correctamente
df_airbnb = df_airbnb.withColumn(
    "last_review", 
    to_date(col("last_review"), "MMMM dd yyyy")
)

df_airbnb.show()


+----------+----------------+-----------+-----+--------------------+--------------------+---------------+
|listing_id|       host_name|last_review|price|         nbhood_full|         description|      room_type|
+----------+----------------+-----------+-----+--------------------+--------------------+---------------+
|      2595|        Jennifer| 2019-05-21|225.0|  Manhattan, Midtown|Skylit Midtown Ca...|entire home/apt|
|      3831|     LisaRoxanne| 2019-07-05| 89.0|Brooklyn, Clinton...|Cozy Entire Floor...|entire home/apt|
|      5099|           Chris| 2019-06-22|200.0|Manhattan, Murray...|Large Cozy 1 BR A...|entire home/apt|
|      5178|        Shunichi| 2019-06-24| 79.0|Manhattan, Hell's...|Large Furnished R...|   private room|
|      5238|             Ben| 2019-06-09|150.0|Manhattan, Chinatown|Cute & Cozy Lower...|entire home/apt|
|      5295|            Lena| 2019-06-22|135.0|Manhattan, Upper ...|Beautiful 1br on ...|entire home/apt|
|      5441|            Kate| 2019-06-23| 85.0

### Actividades

¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

In [0]:
from pyspark.sql.functions import min, max

# Obtener la fecha de la primera reseña
first_review_date = df_airbnb.agg(min("last_review")).collect()[0][0]

# Obtener la fecha de la última reseña
last_review_date = df_airbnb.agg(max("last_review")).collect()[0][0]

# Mostrar las fechas
print(f"Fecha de la primera reseña: {first_review_date}")
print(f"Fecha de la última reseña: {last_review_date}")


Fecha de la primera reseña: 2019-01-01
Fecha de la última reseña: 2019-07-09


¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

In [0]:
# Filtrar las filas donde el tipo de habitación es "Private room" y contar el número de filas
private_room_count = df_airbnb.filter(df_airbnb["room_type"] == "private room").count()

# Mostrar el resultado
print(f"Número de anuncios con habitación privada: {private_room_count}")


Número de anuncios con habitación privada: 11282


¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

In [0]:
from pyspark.sql.functions import avg, round

# Calculamos el precio promedio y redondeamos a dos decimales.
average_price = df_airbnb.agg(round(avg("price"), 2)).collect()[0][0]

# Mostrar el resultado
print(f"El precio promedio de los anuncios es: {average_price}")

El precio promedio de los anuncios es: 141.78


Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
from pyspark.sql import Row

# Creamos la fila guardando los nombres indicados con las variables que creamos anteriormente
row_data = Row(first_reviewed=first_review_date,
               last_reviewed=last_review_date,
               nb_private_rooms=private_room_count,
               avg_price=average_price)

# Creamos el dataframe
review_dates = spark.createDataFrame([row_data])

# Mostramos el dataframe creado
review_dates.display()
          

first_reviewed,last_reviewed,nb_private_rooms,avg_price
2019-01-01,2019-07-09,11282,141.78
